In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.resnext_enhancement as unet
from malaya_speech.train.model import enhancement
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [4]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, (None, 1))
partitioned_x = malaya_speech.tf_featurization.pad_and_partition(x, 4096)
model = unet.Model(partitioned_x, channels_interval = 24)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
down_0 (?, 4096, 24)
Tensor("strided_slice_3:0", shape=(?, 2048, 24), dtype=float32)
down_1 (?, 2048, 48)
Tensor("strided_slice_4:0", shape=(?, 1024, 48), dtype=float32)
down_2 (?, 1024, 72)
Tensor("strided_slice_5:0", shape=(?, 512, 72), dtype=float32)
down_3 (?, 512, 96)
Tensor("strided_slice_6:0", shape=(?, 256, 96), dtype=float32)
down_4 (?, 256, 120)
Tensor("strided_slice_7:0", shape=(?, 128, 120), dtype=float32)
down_5 (?, 128, 144)
Tensor("strided_slice_8:0", shape=(?, 64, 144), dtype=float32)
down_6 (?, 64, 168)
Tensor("strided_slice_9:0", shape=(?, 32, 168), dtype=float32)
down_7 (?, 32, 192)
Tensor("strided_slice_10:0", shape=(?, 16, 192), dtype=float32)
down_8 (?, 16, 216)
Tensor("strided_slice_11:0", shape=(?, 8, 216), dtype=float32)
down_9 (?, 8, 240)
Tensor("strided_slice_12:0", shape=(?, 4, 240), dtype=float32)
down_10 (?, 4, 264)
Tensor("strided_slice_13:0", shape=(?, 2, 264), dtype=float32

In [5]:
model.logits

<tf.Tensor 'sequential_13/activation/Tanh:0' shape=(?, 4096, 1) dtype=float32>

In [6]:
tf.trainable_variables()

[<tf.Variable 'down_0_1_w:0' shape=(1, 1, 1) dtype=float32_ref>,
 <tf.Variable 'down_0_1_b:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_13/gamma:0' shape=(1,) dtype=float32>,
 <tf.Variable 'batch_normalization_13/beta:0' shape=(1,) dtype=float32>,
 <tf.Variable 'down_0_2_w:0' shape=(5, 1, 1) dtype=float32_ref>,
 <tf.Variable 'down_0_2_b:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_14/gamma:0' shape=(1,) dtype=float32>,
 <tf.Variable 'batch_normalization_14/beta:0' shape=(1,) dtype=float32>,
 <tf.Variable 'down_0_3_w:0' shape=(5, 1, 24) dtype=float32_ref>,
 <tf.Variable 'down_0_3_b:0' shape=(24,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_15/gamma:0' shape=(24,) dtype=float32>,
 <tf.Variable 'batch_normalization_15/beta:0' shape=(24,) dtype=float32>,
 <tf.Variable 'down_0_sc_w:0' shape=(1, 1, 24) dtype=float32_ref>,
 <tf.Variable 'down_0_sc_b:0' shape=(24,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_16/gamma:0' shap

In [7]:
logits = tf.reshape(model.logits, (-1, 1))
logits = logits[:tf.shape(x)[0]]
logits

<tf.Tensor 'strided_slice_40:0' shape=(?, 1) dtype=float32>

In [8]:
snr = enhancement.loss.snr(model.logits, partitioned_x)
sdr = enhancement.loss.sdr(model.logits, partitioned_x)

In [9]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [10]:
import numpy as np

In [11]:
y, sr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 22050)

In [12]:
y_, snr_, sdr_ = sess.run([logits, snr, sdr], feed_dict = {x: np.expand_dims(y, -1)})
y_.shape

(258867, 1)

In [13]:
import IPython.display as ipd

ipd.Audio(y_.reshape((-1)), rate = 22050)

In [14]:
snr_, sdr_

((0.52534604, -19.900059), 5.4776456e-07)

In [15]:
y

array([-0.00570545, -0.01051882, -0.01370371, ...,  0.0140948 ,
        0.01236383,  0.01233889])

In [16]:
y_[:,0]

array([ 0.5977475 ,  0.16460711, -0.31104946, ..., -0.28394434,
        0.2573615 ,  0.33466077], dtype=float32)

In [17]:
saver = tf.train.Saver()
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [18]:
!ls -lh test

total 72088
-rw-r--r--  1 huseinzolkepli  staff    77B Feb 10 13:09 checkpoint
-rw-r--r--  1 huseinzolkepli  staff    34M Feb 10 13:09 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    12K Feb 10 13:09 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff   1.0M Feb 10 13:09 model.ckpt.meta


In [19]:
!rm -rf test